# Final Project

## Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
import plotly.graph_objects as go
import panel as pn  
import panel.widgets as pnw
import plotly.graph_objs as go
pn.extension('plotly')

## Read the file and create the DataFrame

In [2]:
debug = False
df = pd.read_csv('conjunto_de_datos_iter_14CSV20.csv')
if(debug):
    # Display the first few rows of the DataFrame
    print(df.head())

municipalities = df['NOM_MUN'].unique().tolist()

# Define age range columns
age_ranges = {
    '0-4': 'P_0A4',
    '5-9': 'P_5A9',
    '10-14': 'P_10A14',
    '15-19': 'P_15A19',
    '20-24': 'P_20A24',
    '25-29': 'P_25A29',
    '30-34': 'P_30A34',
    '35-39': 'P_35A39',
    '40-44': 'P_40A44',
    '45-49': 'P_45A49',
    '50-54': 'P_50A54',
    '55-59': 'P_55A59',
    '60-64': 'P_60A64',
    '65-69': 'P_65A69',
    '70-74': 'P_70A74',
    '75-79': 'P_75A79',
    '80-84': 'P_80A84',
    '85+': 'P_85YMAS'
}

# Assuming the sex data is indicated in a column named 'SEX', adjust if necessary
sex_options = {'Male': '_M', 'Female': '_F'}

## Function for normal distribution

In [10]:
def normal_distribution(datos):
    
  if isinstance(datos, pd.Series):
    datos = datos.to_numpy() # Convert the Series to a NumPy array
   
  
  # Calcular la media y la desviación estándar
  media = np.mean(datos)
  desviacion_estandar = np.std(datos)

  # Generar un rango de valores para el eje x
  rango_x = np.linspace(min(datos) - 10, max(datos) + 10, 1000)

  # Calcular la distribución normal
  distribucion = (1 / (desviacion_estandar * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((rango_x - media) / desviacion_estandar) ** 2)

  return rango_x, distribucion, media, desviacion_estandar

In [11]:
def plot_distribution(selected_municipio, selected_sex):

  ages_columns = ['P_0A4',
                'P_5A9',
                'P_10A14',
                'P_15A19', 
                'P_20A24',
                'P_25A29',
                'P_30A34',
                'P_35A39',
                'P_40A44',
                'P_45A49',
                'P_50A54',
                'P_55A59',
                'P_60A64',
                'P_65A69',
                'P_70A74',
                'P_75A79',
                'P_80A84',
                'P_85YMAS']

 
  # Get the suffix for the selected sex (_M for Male, _F for Female)
  sex_suffix = sex_options[selected_sex]

  # Build the full column name for the selected age range and sex
  for i in range(len(ages_columns)):
    ages_columns[i] = ages_columns[i] + sex_suffix
  
  selected_municipio = [selected_municipio]
  filtered_df = df[df['NOM_MUN'].isin(selected_municipio)]
  total_population = filtered_df.groupby('NOM_MUN')[ages_columns].sum().reset_index()

  
  population = total_population[:1]
  population = population.iloc[0] # Select the first row 
  population = population[1:] #Remove municipio name
  population = population.str.replace('*', '')
  population = pd.to_numeric(population, errors='coerce') 

  rango_x, distribucion, media, desviacion_estandar = normal_distribution(population)
  
  return rango_x, distribucion, media, desviacion_estandar

In [14]:
# Widget definitions
select_graph = pnw.RadioBoxGroup(name='Grafica', width=50, value='Histograma', options=['Histogram', 'Normal distribution'])
select_Municipio = pnw.Select(name='Municipality', width=90, value='Guadalajara', options=municipalities)
select_Municipality = pnw.MultiSelect(name='Municipalities', options=municipalities, size=6)
select_AgeRange = pnw.Select(name='Age Range', width=90, value='0-4', options=list(age_ranges.keys()))
select_Sex = pnw.RadioBoxGroup(name='Sex', value='Male', options=list(sex_options.keys()))

# Function to show population count in a graph for the selected municipalities and age range
# Function to generate the population graph based on user selections
@pn.depends(select_graph, select_Municipality, select_Municipio, select_AgeRange, select_Sex)
def plot_population(select_graph, selected_municipalities, select_municipio, selected_age_range, selected_sex):
    if not selected_municipalities:
        return go.Figure()  # Return an empty figure if no municipalities are selected

    
    if select_graph == 'Histogram':
      # Get the suffix for the selected sex (_M for Male, _F for Female)
      sex_suffix = sex_options[selected_sex]

      # Build the full column name for the selected age range and sex
      age_range_column = age_ranges[selected_age_range] + sex_suffix

      # Ensure the age range column is numeric for summing population values
      df[age_range_column] = pd.to_numeric(df[age_range_column], errors='coerce')

      # Sum population data by municipality
      filtered_df = df[df['NOM_MUN'].isin(selected_municipalities)]
      total_population = filtered_df.groupby('NOM_MUN')[age_range_column].sum().reset_index()
      x_data = total_population['NOM_MUN']
      y_data = total_population[age_range_column]
      x_axistitle = 'Municipality'
      y_axistitle = 'Population'
      title_graph = f'Total Population for Selected Municipalities (Age: {selected_age_range}, Sex: {selected_sex})'

    elif select_graph == 'Normal distribution':
      rango_x, distribucion, media, desviacion_estandar = plot_distribution(select_municipio, selected_sex)
      x_data = rango_x
      y_data = distribucion
      x_axistitle = ' '
      y_axistitle = 'Probability Density'
      title_graph = f'Normal distribution for Selected Municipality'
    # Create a Plotly bar chart
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x = x_data,
        y = y_data,
        #name=f'Age: {selected_age_range} ({selected_sex})',
        marker=dict(color='red')  # Use a default color
    ))

    # Update the layout of the chart
    fig.update_layout(
        title= title_graph,
        xaxis_title= x_axistitle,
        yaxis_title= y_axistitle,
        template='plotly_dark'
    )
    
    return fig

@pn.depends(select_graph)
def conditional_widgets(select_graph):
    if select_graph == 'Histogram':
        return pn.Column(select_Municipality, select_AgeRange, select_Sex)
    elif select_graph == 'Normal distribution':
        return pn.Column(select_Municipio, select_Sex)
    return pn.Spacer()


# Panel layout with widgets and the plot
layout = pn.Column(
    pn.Row(select_graph),
    conditional_widgets,
    pn.Row(plot_population)
)

# Display the layout in a servable application
layout.servable()


Column
    [0] Row
        [0] RadioBoxGroup(name='Grafica', options=['Histogram', ...], width=50)
    [1] ParamFunction(function)
    [2] Row
        [0] ParamFunction(function)